In [ ]:
nltk.download('averaged_perceptron_tagger')

In [280]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
import library as lib
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer

warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_pickle('clean_data_1.pkl')

In [3]:
df.head()

,context,question,answers,plausible_answers,is_impossible
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0


In [4]:
len(df)

130319

In [5]:
test_df = df[:100]

In [7]:
test_df.tail()

,context,question,answers,plausible_answers,is_impossible
95,LeToya Luckett and Roberson became unhappy wit...,Who replaced Luckett and Roberson in Destiny's...,Farrah Franklin and Michelle Williams.,,0
96,LeToya Luckett and Roberson became unhappy wit...,Who was blamed for Luckett and Roberson leavin...,Beyoncé,,0
97,LeToya Luckett and Roberson became unhappy wit...,Who helped Beyoncé overcome her depression dur...,her mother,,0
98,LeToya Luckett and Roberson became unhappy wit...,Which newest member was removed from Destiny's...,Farrah Franklin,,0
99,"The remaining band members recorded ""Independe...","""Charlie's Angels"" featured which single from ...",Independent Women Part I,,0


In [9]:
test_df.context[0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

Inspecting the text in one context instance, we can see that we have to: 1. concatenate lines by removing any \n; 2. tokenize sentences; 3. remove punctuation; 4. lemmatize and stem words; 5. rm stop words

Some of these steps 

Current questions to be answered regarding the structure of the input data:
    1. what to do regarding multiple plausible answers? this will obvs have repercussion on loss fn
    2. how to plug in the wordvectors? easily understandable if they're going in 1-by-1
    3. what preprocessing should be automated inside the learning / prediction steps and what should be carried out outside of that? A: at very least the endline char removal should happen right now. For today, I can test and then refactor the code for the stop word removal. Stemming and lemming can be left out. 
    

In [133]:
test_str = 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [137]:
sent = test_str.translate(str.maketrans('','', string.punctuation)).split(' ')
sent

['Beyoncé',
 'Giselle',
 'KnowlesCarter',
 'biːˈjɒnseɪ',
 'beeYONsay',
 'born',
 'September',
 '4',
 '1981',
 'is',
 'an',
 'American',
 'singer',
 'songwriter',
 'record',
 'producer',
 'and',
 'actress',
 'Born',
 'and',
 'raised',
 'in',
 'Houston',
 'Texas',
 'she',
 'performed',
 'in',
 'various',
 'singing',
 'and',
 'dancing',
 'competitions',
 'as',
 'a',
 'child',
 'and',
 'rose',
 'to',
 'fame',
 'in',
 'the',
 'late',
 '1990s',
 'as',
 'lead',
 'singer',
 'of',
 'RB',
 'girlgroup',
 'Destinys',
 'Child',
 'Managed',
 'by',
 'her',
 'father',
 'Mathew',
 'Knowles',
 'the',
 'group',
 'became',
 'one',
 'of',
 'the',
 'worlds',
 'bestselling',
 'girl',
 'groups',
 'of',
 'all',
 'time',
 'Their',
 'hiatus',
 'saw',
 'the',
 'release',
 'of',
 'Beyoncés',
 'debut',
 'album',
 'Dangerously',
 'in',
 'Love',
 '2003',
 'which',
 'established',
 'her',
 'as',
 'a',
 'solo',
 'artist',
 'worldwide',
 'earned',
 'five',
 'Grammy',
 'Awards',
 'and',
 'featured',
 'the',
 'Billboard',

In [138]:
wordtest = [word.lower() for word in sent]

In [139]:
wordtest

['beyoncé',
 'giselle',
 'knowlescarter',
 'biːˈjɒnseɪ',
 'beeyonsay',
 'born',
 'september',
 '4',
 '1981',
 'is',
 'an',
 'american',
 'singer',
 'songwriter',
 'record',
 'producer',
 'and',
 'actress',
 'born',
 'and',
 'raised',
 'in',
 'houston',
 'texas',
 'she',
 'performed',
 'in',
 'various',
 'singing',
 'and',
 'dancing',
 'competitions',
 'as',
 'a',
 'child',
 'and',
 'rose',
 'to',
 'fame',
 'in',
 'the',
 'late',
 '1990s',
 'as',
 'lead',
 'singer',
 'of',
 'rb',
 'girlgroup',
 'destinys',
 'child',
 'managed',
 'by',
 'her',
 'father',
 'mathew',
 'knowles',
 'the',
 'group',
 'became',
 'one',
 'of',
 'the',
 'worlds',
 'bestselling',
 'girl',
 'groups',
 'of',
 'all',
 'time',
 'their',
 'hiatus',
 'saw',
 'the',
 'release',
 'of',
 'beyoncés',
 'debut',
 'album',
 'dangerously',
 'in',
 'love',
 '2003',
 'which',
 'established',
 'her',
 'as',
 'a',
 'solo',
 'artist',
 'worldwide',
 'earned',
 'five',
 'grammy',
 'awards',
 'and',
 'featured',
 'the',
 'billboard',

In [155]:
def preprocessing(df, columns_list=['context', 'question'], lower_all=False, rm_stopwords=True, lemm=False, stem=False, punct_to_keep=["-"]):
    """ Here we take out unnecessary special characters; tokenize sentences; 
    tokenize within the sentences; remove stop words; add POS tags and, if specified, carry out 
    lemmatization and, where that is not possible, stemming. 
    Lemming and stemming have been left as default: False because we would most likely 
    want to keep the morphological structure of the question and context words intact, 
    given that we are producing predictions that are subsegments of the text span. 
    """
    if stem:
        porter = PorterStemmer()
    if lemm:
        lemmy = WordNetLemmatizer()
    
    stop_words = set(stopwords.words('english'))
    string_mod_punctuation=string.punctuation
    for x in string_mod_punctuation:
        if x in punct_to_keep:
            string_mod_punctuation = string_mod_punctuation.replace( x , "")

    new_df = df.copy()
    
    for col in columns_list:
        new_df[col].astype('str')
        new_df[f'{col}_processed'] = 0
        for i in range(len(new_df[col])):
            final_words=[]
            sentences = nltk.sent_tokenize(new_df[col][i])
            for sentence in sentences: 
                input_words = sentence.translate(str.maketrans('','', string_mod_punctuation)).split(' ')
                words = [word for word in input_words if not word in stop_words]
                words = [word for word in input_words if word != " "]
                if lower_all:
                    words = [word.lower() for word in words]
                tagged_words=nltk.pos_tag(words)
                for w in tagged_words:
                    if lemm:
                        morph_fn = get_morph(w[1])
                        if morph_fn!='':
                            new_w = (lemmy.lemmatize(w[0], morph_fn), w[1])
                        else:
                            new_w = w
                    elif (lemm==False and stem):
                        new_w = (porter.stem(w[0]), w[1])
                    else:
                        new_w=w
                        
                    final_words.append(new_w)
            new_df[f'{col}_processed'][i] = final_words        
    return new_df



def get_morph(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('S'):
        return wordnet.ADJ
    elif pos_tag.startswith('A'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''


In [159]:
t = preprocessing(test_df[:2], lemm=True, lower_all=True)

('beyoncé', 'NN')
('giselle', 'VBD')
('knowles-carter', 'JJ')
('biːˈjɒnseɪ', 'JJ')
('bee-yon-say', 'NN')
('born', 'JJ')
('september', 'NN')
('4', 'CD')
('1981', 'CD')
('american', 'JJ')
('singer', 'NN')
('songwriter', 'NN')
('record', 'NN')
('producer', 'NN')
('actress', 'NN')
('born', 'NNS')
('raised', 'VBD')
('houston', 'NN')
('texas', 'NN')
('performed', 'VBD')
('various', 'JJ')
('singing', 'VBG')
('dancing', 'VBG')
('competitions', 'NNS')
('child', 'NN')
('rose', 'VBD')
('fame', 'JJ')
('late', 'RB')
('1990s', 'CD')
('lead', 'JJ')
('singer', 'NN')
('rb', 'VBD')
('girl-group', 'JJ')
('destinys', 'NN')
('child', 'NN')
('managed', 'VBN')
('father', 'NN')
('mathew', 'NN')
('knowles', 'NNS')
('group', 'NN')
('became', 'VBD')
('one', 'CD')
('worlds', 'JJR')
('best-selling', 'NN')
('girl', 'NN')
('groups', 'NNS')
('time', 'NN')
('their', 'PRP$')
('hiatus', 'NN')
('saw', 'VBD')
('release', 'NN')
('beyoncés', 'NN')
('debut', 'NN')
('album', 'VBZ')
('dangerously', 'RB')
('love', 'JJ')
('2003'

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [180]:
t.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,context,question,answers,plausible_answers,is_impossible,context_processed,question_processed
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec..."
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com..."


In [182]:
len(t.plausible_answers[0])

0

In [161]:
t.context_processed[0]

[('beyoncé', 'NN'),
 ('giselle', 'VBD'),
 ('knowles-carter', 'JJ'),
 ('biːˈjɒnseɪ', 'JJ'),
 ('bee-yon-say', 'NN'),
 ('born', 'JJ'),
 ('september', 'NN'),
 ('4', 'CD'),
 ('1981', 'CD'),
 ('american', 'JJ'),
 ('singer', 'NN'),
 ('songwriter', 'NN'),
 ('record', 'NN'),
 ('producer', 'NN'),
 ('actress', 'NN'),
 ('born', 'NNS'),
 ('raise', 'VBD'),
 ('houston', 'NN'),
 ('texas', 'NN'),
 ('perform', 'VBD'),
 ('various', 'JJ'),
 ('sing', 'VBG'),
 ('dance', 'VBG'),
 ('competition', 'NNS'),
 ('child', 'NN'),
 ('rise', 'VBD'),
 ('fame', 'JJ'),
 ('late', 'RB'),
 ('1990s', 'CD'),
 ('lead', 'JJ'),
 ('singer', 'NN'),
 ('rb', 'VBD'),
 ('girl-group', 'JJ'),
 ('destiny', 'NN'),
 ('child', 'NN'),
 ('manage', 'VBN'),
 ('father', 'NN'),
 ('mathew', 'NN'),
 ('knowles', 'NNS'),
 ('group', 'NN'),
 ('become', 'VBD'),
 ('one', 'CD'),
 ('worlds', 'JJR'),
 ('best-selling', 'NN'),
 ('girl', 'NN'),
 ('group', 'NNS'),
 ('time', 'NN'),
 ('their', 'PRP$'),
 ('hiatus', 'NN'),
 ('saw', 'VBD'),
 ('release', 'NN'),
 ('bey

In [206]:
test_df3=test_df[1:]
test_df3.head()

,context,question,answers,plausible_answers,is_impossible
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0
5,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,,0


In [208]:
test_df3.reset_index(drop=True)

,context,question,answers,plausible_answers,is_impossible
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,,0
...,...,...,...,...,...
94,LeToya Luckett and Roberson became unhappy wit...,Who replaced Luckett and Roberson in Destiny's...,Farrah Franklin and Michelle Williams.,,0
95,LeToya Luckett and Roberson became unhappy wit...,Who was blamed for Luckett and Roberson leavin...,Beyoncé,,0
96,LeToya Luckett and Roberson became unhappy wit...,Who helped Beyoncé overcome her depression dur...,her mother,,0
97,LeToya Luckett and Roberson became unhappy wit...,Which newest member was removed from Destiny's...,Farrah Franklin,,0


In [211]:
test2 = lib.preprocessing(test_df[1:], columns_list=['context','question','answers','plausible_answers'], lemm=True, lower_all=True)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/Users/flatironschool/Desktop/Io/NLG_for_ed_tech/library.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_w=w


In [212]:
test2

,context,question,answers,plausible_answers,is_impossible,context_processed,question_processed,answers_processed,plausible_answers_processed
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (rb, NN), (group, NN), (lead, VBD),...","[(destiny, NN), (child, NN)]",
...,...,...,...,...,...,...,...,...,...
94,LeToya Luckett and Roberson became unhappy wit...,Who replaced Luckett and Roberson in Destiny's...,Farrah Franklin and Michelle Williams.,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(who, WP), (replace, VBD), (luckett, JJ), (ro...","[(farrah, NN), (franklin, NN), (michelle, NN),...",
95,LeToya Luckett and Roberson became unhappy wit...,Who was blamed for Luckett and Roberson leavin...,Beyoncé,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(who, WP), (blame, VBD), (luckett, JJ), (robe...","[(beyoncé, NN)]",
96,LeToya Luckett and Roberson became unhappy wit...,Who helped Beyoncé overcome her depression dur...,her mother,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(who, WP), (help, VBD), (beyoncé, VB), (overc...","[(mother, NN)]",
97,LeToya Luckett and Roberson became unhappy wit...,Which newest member was removed from Destiny's...,Farrah Franklin,,0,"[(letoya, JJ), (luckett, JJ), (roberson, NN), ...","[(which, WDT), (new, JJS), (member, NN), (remo...","[(farrah, NN), (franklin, NN)]",


In [240]:
# test4 = lib.preprocessing(test2, columns_list=['context', 'question', 'answers', 'plausible_answers'],lower_all=True, pos_tagging=False, suffix='simple_preprocessed')
test4 = lib.preprocessing(test2, columns_list=['context','question', 'answers', 'plausible_answers'], lower_all=True, pos_tagging=False, suffix='simple')

In [242]:
test4.context_simple[0]

['beyoncé',
 'giselle',
 'knowles-carter',
 'biːˈjɒnseɪ',
 'bee-yon-say',
 'born',
 'september',
 '4',
 '1981',
 'american',
 'singer',
 'songwriter',
 'record',
 'producer',
 'actress',
 'born',
 'raised',
 'houston',
 'texas',
 'performed',
 'various',
 'singing',
 'dancing',
 'competitions',
 'child',
 'rose',
 'fame',
 'late',
 '1990s',
 'lead',
 'singer',
 'rb',
 'girl-group',
 'destinys',
 'child',
 'managed',
 'father',
 'mathew',
 'knowles',
 'group',
 'became',
 'one',
 'worlds',
 'best-selling',
 'girl',
 'groups',
 'time',
 'their',
 'hiatus',
 'saw',
 'release',
 'beyoncés',
 'debut',
 'album',
 'dangerously',
 'love',
 '2003',
 'established',
 'solo',
 'artist',
 'worldwide',
 'earned',
 'five',
 'grammy',
 'awards',
 'featured',
 'billboard',
 'hot',
 '100',
 'number-one',
 'singles',
 'crazy',
 'love',
 'baby',
 'boy']

In [243]:
test4.head()

,context,question,answers,plausible_answers,is_impossible,context_processed,question_processed,answers_processed,plausible_answers_processed,context_simple,question_simple,answers_simple,plausible_answers_simple
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[what, areas, beyonce, compete, growing]","[singing, dancing]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, leave, destinys, child, become...",[2003],
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, city, state, beyonce, grow]","[houston, texas]",
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, decade, beyonce, become, famous]","[late, 1990s]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (rb, NN), (group, NN), (lead, VBD),...","[(destiny, NN), (child, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, rb, group, lead, singer]","[destinys, child]",


In [213]:
test2.context_processed[1]

[('beyoncé', 'NN'),
 ('giselle', 'VBD'),
 ('knowles-carter', 'JJ'),
 ('biːˈjɒnseɪ', 'JJ'),
 ('bee-yon-say', 'NN'),
 ('born', 'JJ'),
 ('september', 'NN'),
 ('4', 'CD'),
 ('1981', 'CD'),
 ('american', 'JJ'),
 ('singer', 'NN'),
 ('songwriter', 'NN'),
 ('record', 'NN'),
 ('producer', 'NN'),
 ('actress', 'NN'),
 ('born', 'NNS'),
 ('raise', 'VBD'),
 ('houston', 'NN'),
 ('texas', 'NN'),
 ('perform', 'VBD'),
 ('various', 'JJ'),
 ('sing', 'VBG'),
 ('dance', 'VBG'),
 ('competition', 'NNS'),
 ('child', 'NN'),
 ('rise', 'VBD'),
 ('fame', 'JJ'),
 ('late', 'RB'),
 ('1990s', 'CD'),
 ('lead', 'JJ'),
 ('singer', 'NN'),
 ('rb', 'VBD'),
 ('girl-group', 'JJ'),
 ('destiny', 'NN'),
 ('child', 'NN'),
 ('manage', 'VBN'),
 ('father', 'NN'),
 ('mathew', 'NN'),
 ('knowles', 'NNS'),
 ('group', 'NN'),
 ('become', 'VBD'),
 ('one', 'CD'),
 ('worlds', 'JJR'),
 ('best-selling', 'NN'),
 ('girl', 'NN'),
 ('group', 'NNS'),
 ('time', 'NN'),
 ('their', 'PRP$'),
 ('hiatus', 'NN'),
 ('saw', 'VBD'),
 ('release', 'NN'),
 ('bey

In [215]:
# k, now that we've achieved that, let's see how it fares with the full dataset. 

processed_df = lib.preprocessing(df, columns_list=['context', 'question', 'answers', 'plausible_answers'], lower_all=True, lemm=True)

In [216]:
processed_df.head()

,context,question,answers,plausible_answers,is_impossible,context_processed,question_processed,answers_processed,plausible_answers_processed
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",


In [218]:
processed_df.tail()

,context,question,answers,plausible_answers,is_impossible,context_processed,question_processed,answers_processed,plausible_answers_processed
130314,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,,matter,1,"[(the, DT), (term, NN), (matter, NN), (use, VB...","[(physic, NNS), (broadly, RB), (agree, VBD), (...",,"[(matter, NN)]"
130315,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,,Alfvén,1,"[(the, DT), (term, NN), (matter, NN), (use, VB...","[(who, WP), (coin, VBD), (term, NN), (partonic...",,"[(alfvén, NN)]"
130316,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,,Gk. common matter,1,"[(the, DT), (term, NN), (matter, NN), (use, VB...","[(what, WP), (another, DT), (name, NN), (anti-...",,"[(gk, NN), (common, JJ), (matter, NN)]"
130317,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,,a specifying modifier,1,"[(the, DT), (term, NN), (matter, NN), (use, VB...","[(matter, NN), (usually, RB), (need, VBP), (us...",,"[(specify, VBG), (modifier, NN)]"
130318,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,,physics,1,"[(the, DT), (term, NN), (matter, NN), (use, VB...","[(what, WP), (field, NN), (study, NN), (variet...",,"[(physic, NNS)]"


In [228]:
proc_df = processed_df.rename(columns={'context_processed': 'context_pos_lemma', 'question_processed': 'question_pos_lemma', 'answers_processed': 'answers_pos_lemma', 'plausible_answers_processed': 'plausible_answers_pos_lemma'})

In [229]:
proc_df.head()

,context,question,answers,plausible_answers,is_impossible,context_pos_lemma,question_pos_lemma,answers_pos_lemma,plausible_answers_pos_lemma
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",


In [239]:
len(proc_df)

130319

In [244]:
final_proc_df = lib.preprocessing(proc_df, columns_list=['context','question', 'answers', 'plausible_answers'], 
                                  lower_all=True, pos_tagging=False, suffix='simple')

In [245]:
final_proc_df.head()

,context,question,answers,plausible_answers,is_impossible,context_pos_lemma,question_pos_lemma,answers_pos_lemma,plausible_answers_pos_lemma,context_simple,question_simple,answers_simple,plausible_answers_simple
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, start, becoming, popular]","[late, 1990s]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[what, areas, beyonce, compete, growing]","[singing, dancing]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[when, beyonce, leave, destinys, child, become...",[2003],
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, city, state, beyonce, grow]","[houston, texas]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, VBD), (knowles-carte...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[beyoncé, giselle, knowles-carter, biːˈjɒnseɪ,...","[in, decade, beyonce, become, famous]","[late, 1990s]",


In [246]:
pd.to_pickle(final_proc_df, 'preprocessed_data.pkl')

In [283]:
final_proc_df2 = lib.preprocessing(df[0:10], columns_list=['context','question', 'answers', 'plausible_answers'], 
                                  lower_all=True, 
                                  pos_tagging=True,
                                  lemm=True,
                                  suffix='lemma_pos')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/Users/flatironschool/Desktop/Io/NLG_for_ed_tech/library.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[('beyoncé', 'NN'), ('giselle', 'NN'), ('knowlescarter', 'NN'), ('biːˈjɒnseɪ', 'NN'), ('beeyonsay', 'VBP'), ('born', 'JJ'), ('september', 'NN'), ('4', 'CD'), ('1981', 'CD'), ('american', 'JJ'), ('singer', 'NN'), ('songwriter', 'NN'), ('record', 'NN'), ('producer', 'NN'), ('actress', 'NN')]
[('beyoncé', 'NN'), ('giselle', 'NN'), ('knowlescarter', 'NN'), ('biːˈjɒnseɪ', 'NN'), ('beeyonsay', 'VBP'), ('born', 'JJ'), ('september', 'NN'), ('4', 'CD'), ('1981', 'CD'), ('american', 'JJ'), ('singer', 'NN'), ('songwriter', 'NN'), ('record', 'NN'), ('producer', 'NN'), ('actress', 'NN'), ('born', 'NNS'), ('raise', 'VBD'), ('houston', 'NN'), ('texas', 'NN'), ('perform', 'VBD'), ('various', 'JJ'), ('sing', 'VBG'), ('dance', 'VBG'), ('competition', 'NNS'), ('child', 'NN'), ('rise', 'VBD'), ('fame', 'JJ'), ('late', 'RB'), ('1990s', 'CD'), ('lead', 'JJ'), ('singer', 'NN'), ('rb', 'NN'), ('girlgroup', 'NN'), ('destinys', 'VBZ'), ('child', 'NN')]
[('beyoncé', 'NN'), ('giselle', 'NN'), ('knowlescarter', 'N

In [284]:
final_proc_df2.head()

,context,question,answers,plausible_answers,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,plausible_answers_lemma_pos
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",


In [288]:
final_proc_df2.context_lemma_pos[0]

[('beyoncé', 'NN'),
 ('giselle', 'NN'),
 ('knowlescarter', 'NN'),
 ('biːˈjɒnseɪ', 'NN'),
 ('beeyonsay', 'VBP'),
 ('born', 'JJ'),
 ('september', 'NN'),
 ('4', 'CD'),
 ('1981', 'CD'),
 ('american', 'JJ'),
 ('singer', 'NN'),
 ('songwriter', 'NN'),
 ('record', 'NN'),
 ('producer', 'NN'),
 ('actress', 'NN'),
 ('born', 'NNS'),
 ('raise', 'VBD'),
 ('houston', 'NN'),
 ('texas', 'NN'),
 ('perform', 'VBD'),
 ('various', 'JJ'),
 ('sing', 'VBG'),
 ('dance', 'VBG'),
 ('competition', 'NNS'),
 ('child', 'NN'),
 ('rise', 'VBD'),
 ('fame', 'JJ'),
 ('late', 'RB'),
 ('1990s', 'CD'),
 ('lead', 'JJ'),
 ('singer', 'NN'),
 ('rb', 'NN'),
 ('girlgroup', 'NN'),
 ('destinys', 'VBZ'),
 ('child', 'NN'),
 ('manage', 'VBN'),
 ('father', 'NN'),
 ('mathew', 'NN'),
 ('knowles', 'NNS'),
 ('group', 'NN'),
 ('become', 'VBD'),
 ('one', 'CD'),
 ('world', 'NN'),
 ('bestselling', 'VBG'),
 ('girl', 'NN'),
 ('group', 'NNS'),
 ('time', 'NN'),
 ('their', 'PRP$'),
 ('hiatus', 'NN'),
 ('saw', 'VBD'),
 ('release', 'NN'),
 ('beyoncés

In [312]:
idf_mat = lib.get_idf_df(final_proc_df2)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [325]:
idf_mat

,idf_weights
100,1.405465
1981,1.405465
1990s,1.405465
2003,1.405465
actress,1.405465
...,...
what,1.405465
when,1.405465
who,1.405465
world,1.405465


In [331]:
new_test1 = lib.get_tf_idf_col(final_proc_df2)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/Users/flatironschool/Desktop/Io/NLG_for_ed_tech/library.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # rather than just the sum
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [333]:
new_test1.context_lemma_pos_tfidf[0]

[('beyoncé', 'NN', 0.013333333333333334),
 ('giselle', 'NN', 0.018739534774775527),
 ('knowlescarter', 'NN', 0.018739534774775527),
 ('biːˈjɒnseɪ', 'NN', 0.018739534774775527),
 ('beeyonsay', 'VBP', 0.018739534774775527),
 ('born', 'JJ', 0.037479069549551054),
 ('september', 'NN', 0.018739534774775527),
 ('4', 'CD', 0.013333333333333334),
 ('1981', 'CD', 0.018739534774775527),
 ('american', 'JJ', 0.018739534774775527),
 ('singer', 'NN', 0.02666666666666667),
 ('songwriter', 'NN', 0.018739534774775527),
 ('record', 'NN', 0.018739534774775527),
 ('producer', 'NN', 0.018739534774775527),
 ('actress', 'NN', 0.018739534774775527),
 ('born', 'NNS', 0.037479069549551054),
 ('raise', 'VBD', 0.018739534774775527),
 ('houston', 'NN', 0.018739534774775527),
 ('texas', 'NN', 0.018739534774775527),
 ('perform', 'VBD', 0.018739534774775527),
 ('various', 'JJ', 0.018739534774775527),
 ('sing', 'VBG', 0.018739534774775527),
 ('dance', 'VBG', 0.018739534774775527),
 ('competition', 'NNS', 0.01873953477

In [ ]:
final_proc_df2 = lib.get_tf_idf_col(final_proc_df2)

In [250]:
final_proc_df2.to_csv('preprocessed_data2.csv')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,context,question,answers,plausible_answers,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,plausible_answers_lemma_pos
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(actress, NN), (child, NN), (time, NN), (boy,...","[(popular, JJ)]","[(1990s, NNS)]",
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(actress, NN), (child, NN), (time, NN), (boy,...","[(grow, VBG)]","[(dance, VBG)]",
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(actress, NN), (child, NN), (time, NN), (boy,...","[(singer, NN)]","[(2003, CD)]",
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(actress, NN), (child, NN), (time, NN), (boy,...","[(grow, NN)]","[(texas, NN)]",
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(actress, NN), (child, NN), (time, NN), (boy,...","[(famous, JJ)]","[(1990s, NNS)]",


In [252]:
final_proc_df2.context_lemma_pos[0]

[('actress', 'NN'), ('child', 'NN'), ('time', 'NN'), ('boy', 'NN')]

In [334]:
df.head()

,context,question,answers,plausible_answers,is_impossible
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0


In [338]:
final_df = lib.preprocessing(df, columns_list=['context','question', 'answers', 'plausible_answers'], 
                                  lower_all=True, 
                                  pos_tagging=True,
                                  lemm=True,
                                  suffix='lemma_pos')

In [339]:
final_df = lib.get_tf_idf_col(final_df)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/Users/flatironschool/Desktop/Io/NLG_for_ed_tech/library.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [340]:
final_df.to_csv('preprocessed_data2.csv')

In [343]:
pd.to_pickle(final_df, 'preprocessed_data2.pkl')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [342]:
final_df.context_lemma_pos_tfidf[0]

[('beyoncé', 'NN', 0.013333333333333334),
 ('giselle', 'NN', 0.018739534774775527),
 ('knowlescarter', 'NN', 0.018739534774775527),
 ('biːˈjɒnseɪ', 'NN', 0.018739534774775527),
 ('beeyonsay', 'VBP', 0.018739534774775527),
 ('born', 'JJ', 0.02666666666666667),
 ('september', 'NN', 0.013333333333333334),
 ('4', 'CD', 0.013333333333333334),
 ('1981', 'CD', 0.013333333333333334),
 ('american', 'JJ', 0.013333333333333334),
 ('singer', 'NN', 0.02666666666666667),
 ('songwriter', 'NN', 0.013333333333333334),
 ('record', 'NN', 0.013333333333333334),
 ('producer', 'NN', 0.013333333333333334),
 ('actress', 'NN', 0.013333333333333334),
 ('born', 'NNS', 0.02666666666666667),
 ('raise', 'VBD', 0.013333333333333334),
 ('houston', 'NN', 0.013333333333333334),
 ('texas', 'NN', 0.013333333333333334),
 ('perform', 'VBD', 0.013333333333333334),
 ('various', 'JJ', 0.013333333333333334),
 ('sing', 'VBG', 0.013333333333333334),
 ('dance', 'VBG', 0.013333333333333334),
 ('competition', 'NNS', 0.0133333333333